In [1]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.base import BaseEstimator
from sklearn.datasets import make_regression
from sklearn.datasets import make_friedman1

Вставьте код в класс, приведённый ниже. В итоге должен получиться бустинг над деревьями.

In [14]:
class SimpleGB(BaseEstimator):
    def __init__(self, tree_params_dict={}, iters=100, tau=1e-1):
        self.tree_params_dict = tree_params_dict
        self.iters = iters
        self.tau = tau
        
    def fit(self, X_data, y_data):
        self.estimators = []
        curr_pred = 0
        for iter_num in range(self.iters):
            # insert code here
            algo = DecisionTreeRegressor(random_state=5, max_depth = int(self.tree_params_dict['max_depth']))
            algo.fit(X_data, y_data - curr_pred)
            curr_pred += self.tau*algo.predict(X_data)
            self.estimators.append(algo)
    
    def predict(self, X_data):
        res = np.zeros(X_data.shape[0])
        for estimator in self.estimators:
            res += self.tau*estimator.predict(X_data)
        return res

## Проверка качества полученного класса

Можете поиграться с параметрами, посмотрим, у кого самое лучшее качество получится

Сгенерируем данные с линейной зависимостью

In [15]:
X_data, y_data = make_regression(n_samples=1000, noise=10, n_features=10, random_state=42)

In [16]:
algo = SimpleGB(
    tree_params_dict={
        'max_depth':4
    },
    iters=100,
    tau = 0.1
)

In [17]:
np.mean(cross_val_score(algo, X_data, y_data, cv=5, scoring='neg_mean_squared_error'))

-1386.5944340754165

In [18]:
np.mean(cross_val_score(DecisionTreeRegressor(max_depth=4), X_data, y_data, cv=5, scoring='neg_mean_squared_error'))

-7550.7275207710281

In [19]:
np.mean(cross_val_score(LinearRegression(), X_data, y_data, cv=5, scoring='neg_mean_squared_error'))

-95.517114056519929

на реально-линейной зависимости не дотягивает до линейной модели, но уже существенно лучше чем одно дерево.

И аналогично, сгенерируем нелинейную зависимость

In [20]:
X_data, y_data = make_friedman1(n_samples=1000, noise=10, n_features=10, random_state=42)

In [21]:
algo = SimpleGB(
    tree_params_dict={
        'max_depth': 1
    },
    iters=100,
    tau=0.1
)

In [22]:
np.mean(cross_val_score(algo, X_data, y_data, cv=5, scoring='neg_mean_squared_error'))

-105.81605332375698

In [23]:
np.mean(cross_val_score(DecisionTreeRegressor(max_depth=6), X_data, y_data, cv=5, scoring='neg_mean_squared_error'))

-141.57804281228829

In [24]:
np.mean(cross_val_score(LinearRegression(), X_data, y_data, cv=5, scoring='neg_mean_squared_error'))

-108.80318041227227

### Финальный код оценки качества (вам нужно GB реализовать и подобрать хорошие tree_params)

In [35]:
# TODO
parameters = {'tree_params_dict': list({'max_depth': i} for i in range(1, 3)),
              'tau': list(np.linspace(0,0.5,5)), 'iters': range(60,100,20)
             }
clf = GridSearchCV(SimpleGB(), parameters, 'neg_mean_squared_error', )
clf.fit(X_data, y_data)
print(clf.best_params_)
print(np.mean(cross_val_score(clf, X_data, y_data, cv=5, scoring='neg_mean_squared_error')))

{'iters': 60, 'tau': 0.25, 'tree_params_dict': {'max_depth': 1}}
-105.557254072
